# 2. 자연어와 단어의 분산 표현

## 2.1 자연어 처리란
사람이 평소에 쓰는 말을 자연어(Natural Language)라고 한다.  
자연어 처리(Natural Language Processing, NLP)를 문자 그대로 해석하면 자연어를 처리하는 분야라고 말한다.

## 2.1.1 단어의 의미
컴퓨터에게 '단어의 의미'를 이해시키는 방법에는 크게 3가지 기법이 있다.  
1. 시소러스를 활용한 기법  
2. 통계 기반 기법  
3. 추론 기반 기법

## 2.2 시소러스
시소러스 방식은 단어의 의미를 직접적으로 나타내는 사전과 같은 방식이 아닌 유의어 사전으로, 뜻이 같은단어(동의어)나 뜻이 비슷한단어(유의어)로 구성되어 있다

## 2.2.1 WordNet
자연어 처리 분야에서 가장 유명한 시소러스는 WordNet이다.  
WordNet은 프린스턴 대학교에서 1985년부터 구축한 시소러스로 지금까지 많은 연구와 다양한 자연어 처리 애플리케이션에서 활용되고 있다.  


## 2.2.2 시소러스의 문제점
WordNet과 같은 시소러스에는 수많은 단어에 대한 동의어와 계층 구조 등의 관계가 정의되어 있는데, 사람이 수작업으로 레이블링 하는 방식에는 큰 결점이 존재한다.  
#### 시대 변화에 대응하기 어렵다.  
단어의 뜻은 새로 생기거나 없어지기 때문에 시소러스의 문제 중 하나이다.  
#### 사람을 쓰는 비용이 크다.  
단어들 모두에 대해 단어 사이의 관계를 정의 해주어야 하는데 영어의 경우 한 단어에 비슷한 뜻은 약 1000만개 정도 된다.  
#### 단어의 미묘한 차이를 표현할 수 없다.  
비슷한 단어끼리 묶여있지만 비슷한 단어라도 미묘한 차이가 있는데 이 차이를 표현할 수 없다.  

시소러스의 문제를 해결하기 위해 통계 기반 기법과 신경망을 사용한 추론 기반 기법을 활용하면 단어의 의미를 자동으로 추출한다.  

## 2.3 통계 기반 기법
통계 기반 기법은 말뭉치(corpus 복수형, 단수 corpora)를 이용한다.

## 2.3.1 파이썬으로 말뭉치 전처리 하기
#### 1. Preprocessing
텍스트 데이터를 단어로 분할하고 그 분할된 단어들을 단어 ID목록으로 변환한다.

In [3]:
text = 'You say goodbye and I say hello.'

text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [4]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

단어에 ID를 부여하고, ID의 리스트로 이용가능하게 해준다.(딕셔너리 사용)

In [5]:
word_to_id = {}

for word in words:
    if word not in word_to_id:
        new_id =  len(word_to_id)
        word_to_id[word] = new_id
        
id_to_word = {id_: word for word, id_ in word_to_id.items()}

In [6]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [7]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [8]:
id_to_word[1]

'say'

In [9]:
word_to_id['hello']

5

In [10]:
# 단어 목록 -> 단어 ID 목록으로 변경
import numpy as np

corpus = [word_to_id[word] for word in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [14]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    for word in words:
        if word not in word_to_id:
            new_id =  len(word_to_id)
            word_to_id[word] = new_id
            
    id_to_word = {id_: word for word, id_ in word_to_id.items()}
    
    corpus = np.array([word_to_id[word] for word in words])
    return corpus, word_to_id, id_to_word

In [15]:
# common/util.py -> preprocess 메서드 사용
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [16]:
corpus, word_to_id, id_to_word

(array([0, 1, 2, 3, 4, 1, 5, 6]),
 {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'})

## 2.3.2 단어의 분산표현
단어의 의미를 정확하게 파악할 수 있는 벡터 표현을 자연어 처리 분야에서는 단어의 분산표현(distributional representation)이라고 한다.

## 2.3.3 분포 가설
단어의 의미는 주변단어에 의해 형성된다라는 것이 분포 가설(distributional hypothesis)라고 하며, 단어를 벡터로 표현하는 최근 연구도 대부분 이 가설에 기초한다.

##  2.3.4 동시발생 행렬(Co-occurrence Matrix)
주변 단어를 세어 보는 방법  
특정 단어에 대해, 그 단어의 주변에 어떤 단어가 몇 번이나 등장하는지 카운팅하여 합치는 방법

In [19]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(f'corpus: {corpus}')
print(f'id_to_word: {id_to_word}')

corpus: [0 1 2 3 4 1 5 6]
id_to_word: {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [20]:
C = np.array([[0, 1, 0, 0, 0, 0, 0], 
              [1, 0, 1, 0, 1, 1, 0], 
              [0, 1, 0, 1, 0, 0, 0], 
              [0, 0, 1, 0, 1, 0, 0], 
              [0, 1, 0, 1, 0, 0, 0], 
              [0, 1, 0, 0, 0, 0, 1], 
              [0, 0, 0, 0, 0, 1, 0]], dtype=np.int32)

In [21]:
# ID가 0인 단어의 벡터 표현
print(C[0])
# ID가 4인 단어의 벡터 표현
print(C[4])

# "goodbye"의 벡터 표현
print(C[word_to_id['goodbye']])

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


In [22]:
# common/util.py
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i  # left window_size
            right_idx = idx + i  # right window_size

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

In [23]:
window_size = 1
vocab_size = len(id_to_word)

C = create_co_matrix(corpus, vocab_size, window_size=1)
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]])

## 2.3.5 벡터 간 유사도
벡터 사이의 유사도를 측정하는 방법은 다양한데, 대표적으로 벡터의 내적이나 유클리드 거리 등을 꼽을 수 있다.  
단어 벡터의 유사도를 나타낼 때는 코사인유사도(cosine similarity)를 자주 이용한다.  
두방향이 완전히 같다면 1을 나타내고, 정 반대이면 -1을 나타낸다.